In [7]:
import obspy
import obspy.signal.filter
from obspy.io.sac.sactrace import SACTrace
from obspy.io.sac.util import get_sac_reftime
import os
import glob
import numpy as np
import shutil
import pickle
from obspy import read

# Convert to envelope, normalize and cut waveform

In [2]:
data_sac = "/home/youzhi/python-projects/sp-project/BAD_SAC_for_readin"

for year in sorted(os.listdir(data_sac)):
    print(year)
    inpath = os.path.join(data_sac, year)
    if os.path.isdir(inpath):
        #print(inpath)
        os.chdir(inpath)
        for bp in sorted(glob.glob("*.bp")):
            #print(bp)
            # calculate the envelope
            tr = obspy.read(bp)
            trenv = obspy.signal.filter.envelope(tr[0].data)
            # normalize the envelope
            trenv = trenv / np.max(trenv)
            tr[0].data = trenv
            tr[0].write(bp +".ev", format='SAC')
            
            nbp = "{}.{}".format(bp,"ev")
            ntr = obspy.read(nbp)
            # cut the waveform within a given time window, -2 to 11 s
            # changed to 11 based on data I collected
            T = get_sac_reftime(ntr[0].stats.sac) + 0
            ntr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            ntr[0].write(nbp, format='SAC')

            tr = obspy.read(bp)
            # cut the waveform within a given time window, -2 to 11 s
            T = get_sac_reftime(tr[0].stats.sac) + 0
            tr.trim(starttime = T-2, endtime = T+11, fill_value=0)
            tr[0].write(bp, format='SAC')

20220112021208390
20220112090603170
20220113032108850
20220113032453520
20220113082309790
20220115161902000
20220116091814410
20220116124930000
20220116145950590
20220117000229990
20220117155620910
20220118073915120
20220118084437210
20220120095230400
20220122010142300
20220122234305360
20220123044939870
20220123205715240
20220124023838010
20220125172319330
20220125192818690
20220125212130440
20220126050555330
20220126095905010
20220126103830020
20220127033401460
20220127095426310
20220128104007860
20220128203303970
20220129055641000
20220131234528720
20220201012424220
20220201192630930
20220202070130500
20220202100105740
20220203075810850
20220203151832540
20220203170822680
20220204105219770
20220205074104630
20220205074909660
20220205183420810
20220205234904960
20220206232447510


# Move ev files into a new folder

In [3]:
target_dir = "/home/youzhi/python-projects/sp-project/envelope_normalized"
# move all the ev files into a folder
data_sac = "/home/youzhi/python-projects/sp-project/BAD_SAC_for_readin"
num_files = 0
for path in os.listdir(data_sac):
    #print(path)
    if path.endswith("ini") != True:
        
        for file in os.listdir(os.path.join(data_sac, path)):
            #print(file)
            if file.endswith(".ev"):
                filepath = os.path.join(data_sac, path, file)
                shutil.copy(filepath, os.path.join(target_dir, file))
                #print(filepath)
	    #some times if might count desktop.ini files
                num_files += 1
print(num_files)

4441


# Combine envelope for neural network readin

In [5]:
datadir = target_dir

signal_evid = []
signal_evlo = []
signal_evla = []
signal_stlo = []
signal_stla = []
signal_fname = []
envelop_signal = []
PmP_label = []
PmP_time = []
PmP_ptime = []
PmP_dist = []
PmP_evdp = []
PmP_mag = []
j=0
for file in os.listdir(datadir):
    if file.endswith(".ev"):
        j=j+1
        st = read(os.path.join(f"{datadir}", file))
        envelop_signal.append(st[0].data)
        filenm = file.split('.',6)
        filenmall=f"{filenm[0]}"+'.'+f"{filenm[1]}"+'.'+f"{filenm[2]}"
        signal_evid.append(st[0].stats.sac.nevid)
        signal_evlo.append(st[0].stats.sac.evlo)
        signal_evla.append(st[0].stats.sac.evla)
        signal_stlo.append(st[0].stats.sac.stlo)
        signal_stla.append(st[0].stats.sac.stla)
        signal_fname.append(filenmall)
        if st[0].stats.sac.get("kt2") != None:
            if st[0].stats.sac.kt2 == "sPmP":
                PmP_label.append(0) # 0 for bad, 1 for good
            else:
                PmP_label.append(0)
        else:
            PmP_label.append(0)
        PmP_time.append(st[0].stats.sac.t2)       ### differential traveltime between P and PmP waves
        PmP_ptime.append(st[0].stats.sac.o)       ### (negative) traveltime of first P wave (This is because waveform has been aligned to P-wave arrival)
        PmP_dist.append(st[0].stats.sac.dist)
        PmP_evdp.append(st[0].stats.sac.evdp)
        PmP_mag.append(st[0].stats.sac.mag)
        #print("%d  %s  %g  %g  %g  %g  %g %d"%(j,signal_fname[j-1],PmP_time[j-1],PmP_ptime[j-1],PmP_dist[j-1],PmP_evdp[j-1],PmP_mag[j-1],PmP_label[j-1]))

# save into a single file

In [8]:
#fast save and retrieve data, no need to read in data again(very slow)

def saveVariables(variables, filename): #where 'variables' is a list of variables
    with open(filename, 'wb+') as file:
        pickle.dump(variables, file)

#save the data
saveVariables([envelop_signal, PmP_time, PmP_label, PmP_dist, PmP_evdp, PmP_mag, signal_stlo, signal_stla, signal_evlo, signal_evla, signal_evid, signal_fname], f"{datadir}/readindata")